<a href="https://colab.research.google.com/github/psshri/Pan-IIT-Hackathon-2019/blob/master/final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!ls

drive  sample_data


In [3]:
from zipfile import ZipFile
file_name = "drive/My Drive/Colab Notebooks/data/training.5k.zip"

with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('done')

done


In [0]:
from tqdm import tqdm

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import natsort
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Flatten, Dropout
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator
from sklearn import model_selection
from keras.models import load_model
from sklearn.preprocessing import LabelBinarizer
from keras.callbacks import ModelCheckpoint
from keras.layers.advanced_activations import LeakyReLU

Using TensorFlow backend.


In [0]:
TRAIN_DIR = 'training/training'

In [0]:
solution = pd.read_csv('training/solution.csv')

In [8]:
IMG_SIZE=240
train_data_240=[]
labels_240=[]
for img in tqdm(os.listdir(TRAIN_DIR)):
    path = os.path.join(TRAIN_DIR,img)
    img_data = cv2.imread(path)
    img_data = cv2.resize(img_data, (IMG_SIZE,IMG_SIZE))
    train_data_240.append([np.array(img_data)])
    number = int(img[:-4])
    categ_num = int(solution[solution.id==number].category)
    labels_240.append(categ_num)

100%|██████████| 5000/5000 [00:17<00:00, 286.11it/s]


In [0]:
lb=LabelBinarizer()
labels_240=lb.fit_transform(labels_240)

In [0]:
x_train,x_test,y_train,y_test=model_selection.train_test_split(train_data_240,labels_240,test_size=0.25,random_state=42)

In [0]:
aug=ImageDataGenerator(rotation_range=25,width_shift_range=0.1,height_shift_range=0.1,shear_range=0.2,horizontal_flip=True,fill_mode="nearest")

In [0]:
model=Sequential()
model.add(Conv2D(32,(3,3),padding="same",activation="linear",input_shape=(240,240,3)))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Dropout(0.25))

model.add(Conv2D(32,(3,3),padding="same",activation="linear"))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(Conv2D(64,(3,3),padding="same",activation="linear"))
model.add(LeakyReLU(alpha=0.1))
model.add(Dropout(0.25))

model.add(Conv2D(128,(3,3),padding="same",activation="linear"))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(128,(3,3),padding="same",activation="relu"))
model.add(BatchNormalization(axis=-1))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(1024,activation="linear"))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(6,activation="softmax"))


In [0]:
model.compile(loss=keras.losses.categorical_crossentropy,optimizer=keras.optimizers.Adam(),metrics=['accuracy'])

In [0]:
x_train = np.array([i[0] for i in x_train]).reshape(-1,240,240,3)
x_test = np.array([i[0] for i in x_test]).reshape(-1,240,240,3)

In [0]:
x_train = x_train/255.0
x_test = x_test/255.0

In [16]:
train=model.fit_generator(aug.flow(x_train,y_train,batch_size=64),validation_data=(x_test,y_test),steps_per_epoch=len(x_train)/64,epochs=150,verbose=1)

Epoch 1/150
59/58 [==============================] - 58s 991ms/step - loss: 2.9007 - acc: 0.1954 - val_loss: 2.0512 - val_acc: 0.2376
Epoch 2/150
59/58 [==============================] - 52s 874ms/step - loss: 2.1890 - acc: 0.2299 - val_loss: 1.9627 - val_acc: 0.2504
Epoch 3/150
59/58 [==============================] - 51s 860ms/step - loss: 2.0862 - acc: 0.2364 - val_loss: 1.7919 - val_acc: 0.2656
Epoch 4/150
59/58 [==============================] - 51s 862ms/step - loss: 2.0041 - acc: 0.2576 - val_loss: 2.2376 - val_acc: 0.2784
Epoch 5/150
59/58 [==============================] - 52s 878ms/step - loss: 1.8468 - acc: 0.2584 - val_loss: 3.6280 - val_acc: 0.2504
Epoch 6/150
59/58 [==============================] - 51s 869ms/step - loss: 1.8495 - acc: 0.2652 - val_loss: 1.9607 - val_acc: 0.2880
Epoch 7/150
59/58 [==============================] - 53s 892ms/step - loss: 1.7873 - acc: 0.2752 - val_loss: 1.9540 - val_acc: 0.2664
Epoch 8/150
59/58 [==============================] - 53s 896ms